# Import .mat file GT info

* Related Page link: http://people.rennes.inria.fr/Ahmet.Iscen/diffusion.html

* README: http://people.rennes.inria.fr/Ahmet.Iscen/readme_eval.txt

In [ ]:
import scipy.io
import numpy as np
from pprint import pprint

mat = scipy.io.loadmat('gnd_instre.mat')
db_image_paths = mat['imlist'][0].tolist()
db_image_paths = list(map(lambda x: str(x)[2:-2], db_image_paths))
pprint(db_image_paths[:5]) # Names of DB images
print(len(db_image_paths))

query_image_paths = mat['qimlist'][0].tolist()
query_image_paths = list(map(lambda x: str(x)[2:-2], query_image_paths))
pprint(query_image_paths[:5]) # Names of query images. 5 random queries were selected from each class, and were excluded from the dataset.
print(len(query_image_paths))

gnds = mat['gnd'][0].tolist()
query_positive_images = []
query_bbox_list = []
for gnd in gnds:
    image_ids = (gnd[0][0]-1).tolist() # convert 1-based indexing (matlab style) to 0-based indexing (python style)
    bbox = gnd[1][0].tolist()
    
    query_positive_images.append(image_ids)
    query_bbox_list.append(bbox)

# query_positive_images = list(map(lambda x: x[0].reshape(None).tolist(), query_positive_images))
print(query_positive_images[:5]) # This is the main structure used for evaluation. Each index contains positive images for the corresponding query, and the bounding box for the 			query object on the query image. 
# 		For example:
# 			gnd(1).ok : Positive images for the first query.
#			gnd(1).bbx: Coordinates of the bounding box containing the query object from qimlist{1}.
pprint(query_bbox_list[:5])

In [ ]:
# Visualize Image

query_list = list(zip(query_image_paths, query_bbox_list, query_positive_images))
db_image_paths = db_image_paths

import os
IMAGE_DIR = "./data/INSTRE"

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def get_image(image_name, crop_bbox=None):
    image_path = os.path.join(IMAGE_DIR, image_name)
    # print("open image:", image_path)
    img_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)        
    if img_bgr is None:
        print("cannot find image with name:", image_name)
        return        
    
    
    if crop_bbox:
        x_st = crop_bbox[0]
        y_st = crop_bbox[1]
        x_en = crop_bbox[2]
        y_en = crop_bbox[3]
        crop_img_bgr = img_bgr[y_st:y_en, x_st:x_en]                
        img_bgr = crop_img_bgr
        
    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        
    return img

for image_name, bbox, pos_image_ids in query_list[0:]:
#     if not "INSTRE-M" in image_name:
#         continue
    print("QUERY image_name:", image_name)    
    print("bbox:", bbox)    
    img = get_image(image_name)
    plt.figure()
    plt.imshow(img)
    plt.show()
    
    img_crop = get_image(image_name, bbox)
    plt.figure()
    plt.imshow(img_crop)
    plt.show()
    
     
    print("RESULT:")
    rank_list = pos_image_ids
    for rank, id in enumerate(rank_list[:10]):
        filename = db_image_paths[id]
        img = get_image(filename)
        score = -1
        
        if rank % 5 == 0:
            plt.show()
            fig = plt.figure(figsize=(20, 5))            
        ax = fig.add_subplot(int("15{}".format(rank%5+1)))
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(img)      
        print("   top{}: ({}) {}. score: {}".format(rank+1, id, filename, score))
    print()
    
    break

plt.show()

In [ ]:
# Query Visualization

for rank, tup in enumerate(query_list[:]):
    image_name, bbox, pos_image_ids = tup
#     if not "INSTRE-M" in image_name:
#         continue
#     print("QUERY image_name:", image_name)    
#     print("bbox:", bbox)    
#     img = get_image(image_name)
#     plt.figure()
#     plt.imshow(img)
#     plt.show()
    
    filename = image_name
    img = get_image(filename, bbox)
    score = -1

    if rank % 5 == 0:
        plt.show()
        fig = plt.figure(figsize=(20, 5))            
    ax = fig.add_subplot(int("15{}".format(rank%5+1)))
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(img)      
    print("   ({}) {}. score: {}".format(rank, filename, score))


plt.show()

# Create Oxf5k, Par6k style GT files

In [ ]:
# Query GT Generation

OUTPUT_DIR = "./gt"
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

for qid, tup in enumerate(query_list[:]):
    image_name, bbox, pos_image_ids = tup
#     if not "INSTRE-M" in image_name:
#         continue
    query_name=image_name.split("/")[1]
    query_filename = "{qid:04d}_{query_name}_query.txt".format(qid=qid, query_name=query_name)
    ok_filename = "{qid:04d}_{query_name}_ok.txt".format(qid=qid, query_name=query_name)
    with open(os.path.join(OUTPUT_DIR, query_filename), "w") as f:
        f.write("{} {} {} {} {}".format(image_name, bbox[0], bbox[1], bbox[2], bbox[3]))
        
    with open(os.path.join(OUTPUT_DIR, ok_filename), "w") as f:
        for rank, id in enumerate(pos_image_ids):
            filename = db_image_paths[id]
            f.write(filename+"\n")
